In [5]:
import os
import io
import surprise
from surprise import model_selection, Dataset

In [3]:
def read_item_names():
    """
    返回 电影名臣=> 电影id  电影id => 电影名称的映射
    """
    file_name = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
    
    # 存放电影id到电影名称的映射
    rid_to_name = {}
    # 存放电影名称到id的映射
    name_to_rid = {}
# 1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0

# 2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0

# 3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f.readlines():
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

In [8]:
data = Dataset.load_builtin('ml-100k')
train_data = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = surprise.KNNBaseline(sim_options=sim_options)
algo.train(train_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [9]:
# 获取电影名到电影id 和 电影id到电影名的映射
rid_to_name, name_to_rid = read_item_names()

In [10]:
# 拿出来Toy Story这部电影对应的item id
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_raw_id

'1'

In [14]:
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)
toy_story_inner_id

24

In [15]:
# 找到最近的10个邻居
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)
toy_story_neighbors

[433, 101, 302, 309, 971, 95, 26, 561, 816, 347]

In [19]:
# 将得到的iid还原成电影名称
for iid in toy_story_neighbors:
    raw_id = algo.trainset.to_raw_iid(iid)
    if raw_id in rid_to_name:
        name = rid_to_name[raw_id]
        print(name, '\n')

Beauty and the Beast (1991) 

Raiders of the Lost Ark (1981) 

That Thing You Do! (1996) 

Lion King, The (1994) 

Craft, The (1996) 

Liar Liar (1997) 

Aladdin (1992) 

Cool Hand Luke (1967) 

Winnie the Pooh and the Blustery Day (1968) 

Indiana Jones and the Last Crusade (1989) 

